In [1]:
import wrds 
import pandas as pd

In [2]:
conn = wrds.Connection() #connect to WRDS servers

Loading library list...
Done


In [3]:
conn.list_libraries().sort()
conn.list_tables(library='optionm') # open the optionmetrics database, theres 2 data sets we need, opprcd2020 and option_price_2020. 
                                    # the latter has the underlying price for the option trading on that day, so we need to merge them
                                    # based off of their optionid and the date, columns available in both datasets

['country',
 'currency',
 'distrd',
 'distribution',
 'distribution_projection',
 'distrprojd1996',
 'distrprojd1997',
 'distrprojd1998',
 'distrprojd1999',
 'distrprojd2000',
 'distrprojd2001',
 'distrprojd2002',
 'distrprojd2003',
 'distrprojd2004',
 'distrprojd2005',
 'distrprojd2006',
 'distrprojd2007',
 'distrprojd2008',
 'distrprojd2009',
 'distrprojd2010',
 'distrprojd2011',
 'distrprojd2012',
 'distrprojd2013',
 'distrprojd2014',
 'distrprojd2015',
 'distrprojd2016',
 'distrprojd2017',
 'distrprojd2018',
 'distrprojd2019',
 'distrprojd2020',
 'distrprojd2021',
 'exchange',
 'exchgd',
 'forward_price',
 'futures',
 'futures_price',
 'fwdprd1996',
 'fwdprd1997',
 'fwdprd1998',
 'fwdprd1999',
 'fwdprd2000',
 'fwdprd2001',
 'fwdprd2002',
 'fwdprd2003',
 'fwdprd2004',
 'fwdprd2005',
 'fwdprd2006',
 'fwdprd2007',
 'fwdprd2008',
 'fwdprd2009',
 'fwdprd2010',
 'fwdprd2011',
 'fwdprd2012',
 'fwdprd2013',
 'fwdprd2014',
 'fwdprd2015',
 'fwdprd2016',
 'fwdprd2017',
 'fwdprd2018',
 'fwdprd

In [8]:
optionsdata2020 = conn.raw_sql("""select symbol, strike_price, optionid, impl_volatility, open_interest, volume, 
                          vega, best_bid, best_offer, date, exdate
                          
                          from optionm.opprcd2020
                          
                          where volume > 500
                          and volume > 0.5 * open_interest
                          and cp_flag = 'C' 
                          and (exdate - date) <= 30 and (exdate-date) > 0 
                          and best_bid > 0.3
                          
                          LIMIT 5""")
                          

In [13]:
optionsdata2020

,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate
0,DLTR 200918C110000,110000.0,135066529.0,0.418918,1879.0,2438.0,8.885530,1.74,1.80,2020-08-24,2020-09-18
1,DLTR 200828C105000,105000.0,135001308.0,0.934839,1538.0,781.0,3.619184,2.38,2.53,2020-08-25,2020-08-28
2,DLTR 200828C103000,103000.0,135066515.0,1.183688,458.0,695.0,3.028524,4.25,4.30,2020-08-26,2020-08-28
3,DLTR 200828C104000,104000.0,135066516.0,1.212922,280.0,753.0,3.070897,3.80,3.90,2020-08-26,2020-08-28
4,DLTR 200828C105000,105000.0,135001308.0,1.227962,1646.0,1888.0,3.076746,3.40,3.45,2020-08-26,2020-08-28


In [18]:
optiondata2 = conn.raw_sql("""select *
                              from optionm.option_price_2020
                              LIMIT 5""")
optiondata2

,securityid,optionid,exchange,currency,bid,underlyingbid,ask,underlyingask,last,underlyinglast,impliedvolatility,delta,gamma,vega,theta,calculationprice,volume,openinterest,specialsettlement,referenceexchange,amsettlement,contractsize,expiryindicator,date,bidtime,asktime,lasttime
0,500004.0,155858312.0,126.0,814.0,-99.989998,47.34,-99.989998,47.34,2.56,47.34,0.195858,0.543403,0.065316,12.54768,-2.720176,S,0.0,1.0,0.0,1058.0,0.0,100.0,None,2020-01-02,0.0,0.0,0.0
1,500004.0,155858314.0,126.0,814.0,-99.989998,47.34,-99.989998,47.34,2.77,47.34,0.189939,-0.509493,0.063797,12.57794,-2.680460,S,0.0,0.0,0.0,1058.0,0.0,100.0,None,2020-01-02,0.0,0.0,0.0
2,500004.0,155858316.0,126.0,814.0,-99.989998,47.34,-99.989998,47.34,2.05,47.34,0.192711,0.476425,0.066891,12.55571,-2.706265,S,0.0,1.0,0.0,1058.0,0.0,100.0,None,2020-01-02,0.0,0.0,0.0
3,500004.0,155858318.0,126.0,814.0,-99.989998,47.34,-99.989998,47.34,3.31,47.34,0.184747,-0.576170,0.064072,12.28616,-2.565836,S,0.0,0.0,0.0,1058.0,0.0,100.0,None,2020-01-02,0.0,0.0,0.0
4,500004.0,155858320.0,126.0,814.0,-99.989998,47.34,-99.989998,47.34,1.63,47.34,0.189682,0.410522,0.065953,12.30581,-2.591786,S,0.0,0.0,0.0,1058.0,0.0,100.0,None,2020-01-02,0.0,0.0,0.0


In [ ]:
finaldata = conn.raw_sql("""select a.symbol, a.strike_price, a.optionid, a.impl_volatility, a.open_interest, a.volume, 
                          a.vega, a.best_bid, a.best_offer, a.date, a.exdate, b.underlyingbid, b.underlyingask 
                          
                          
                          from optionm.opprcd2020 a
                          inner join optionm.option_price_2020 b
                          
                          on a.optionid = b.optionid
                          and a.date = b.date
                          
                          
                          where a.volume > 500
                          and a.volume > 0.5 * a.open_interest
                          and a.cp_flag = 'C' 
                          and (a.exdate - a.date) <= 30 and (a.exdate-a.date) > 0 
                          and a.best_bid > 0.3   
                          and (1 - ((b.underlyingbid+b.underlyingask)/2)/(a.strike_price/1000)) > .05
                          LIMIT 50""") # this is the part that doesn't seem to work